In [11]:
import polars as pl
import re

In [3]:
# data download
missing_symbols = []

with open("../log/data_download.py.log", "r") as f:
    
    for line in f:

        if line.startswith("$"):

            symbol = re.search(r"[\d, A-Z]{4}\.T", line)
            if symbol: missing_symbols.append(symbol.group(0)[:-2])

df = pl.read_csv("../resources/all_stock.csv")
missing_details = []

for symbol in missing_symbols:

    missing_details.append(df.filter(pl.col("コード") == symbol).to_dicts()[0])

missing_df = pl.DataFrame(missing_details)
missing_df.write_csv("../resources/missing_stock.csv")

In [ ]:
# data filtering
discarded_stock_symbols = {}
discarded_trust_symbols = {}

with open("../log/data_filtering.py.log", "r") as f:

    for line in f:

        if line.startswith(">"):

            symbol = re.search(r">[\d, A-Z]{4},", line)
            reason = line.split(", ")[1][:-1]

            if symbol:
                
                discarded_stock_symbols[symbol.group(0)[1:-1]] = reason

            else:

                symbol = re.search(r">[\d, A-Z]{8},", line)
                if symbol:
                    
                    discarded_trust_symbols[symbol.group(0)[1:-1]] = reason

df = pl.read_csv("../resources/all_stock.csv")
missing_stock_details = []

for symbol, reason in discarded_stock_symbols.items():

    entry = df.filter(pl.col("コード") == symbol).to_dicts()[0]
    entry["理由"] = reason

    missing_stock_details.append(entry)

missing_stock_df = pl.DataFrame(missing_stock_details)
missing_stock_df.write_csv("../resources/discarded_stock.csv")

df = pl.read_csv("../resources/all_trust.csv")
missing_trust_details = []

for symbol, reason in discarded_trust_symbols.items():

    entry = df.filter(pl.col("協会コード") == symbol).to_dicts()[0]
    entry["理由"] = reason

    missing_trust_details.append(entry)

missing_trust_df = pl.DataFrame(missing_trust_details)
missing_trust_df.write_csv("../resources/discarded_trust.csv")

: 